# Tarea 2

Diego Lozoya Morales | 745345

In [12]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.optimize import minimize
from IPython.display import display

pd.set_option('display.float_format', '{:,.2f}'.format)

## Ejercicio 1 

Función VaR y ES

In [13]:
def VaR_CVaR(data, n_stocks, conf, long):
     
     valor_posicion = n_stocks * data.iloc[-1]

     weights = []
     for i in range(len(n_stocks)):
          weights.append(valor_posicion.iloc[i]/valor_posicion.sum())
          
     rt_assets = data.pct_change().dropna()

     rt_port = rt_assets.mul(weights).sum(axis=1)

     if long:
          var = np.percentile(rt_port,100-conf)
          cvar = rt_port[rt_port < var].mean()
     else:
          var = np.percentile(rt_port,conf)
          cvar = rt_port[rt_port > var].mean()

     var_m = var * valor_posicion.sum()
     cvar_m = cvar * valor_posicion.sum()

     resultados_df = pd.DataFrame({
        "Métrica": ["VaR", "CVaR"],
        "Porcentual (%)": [np.round(var * 100, 4), np.round(cvar * 100, 4)],
        "Monetario ($)": [np.round(var_m, 2), np.round(cvar_m, 2)]
    })

     return resultados_df

In [14]:
data = yf.download(['AAPL', 'AMD', 'JPM', 'LMT', 'TSLA'], start="2020-01-01", end="2023-01-01")['Close'] #No puedo descargar el ajustado

positions = [2193, 3221, 1231, 761, 1211] #Ordenar alfabeticamente

long = True #True o False

conf = 95 #[0, 100]

VaR_CVaR(data, positions, conf, long)

[*********************100%***********************]  5 of 5 completed


,Métrica,Porcentual (%),Monetario ($)
0,VaR,-2.88,"-33,073.87"
1,CVaR,-4.63,"-53,131.45"


In [15]:
VaR_CVaR(data, positions, conf, long)["Monetario ($)"][0]

-33073.87

## Ejercicio 2

Rebalanceo del portafolio

In [16]:
rt = data.pct_change().dropna()
position_value = positions * data.iloc[-1]
portfolio_value = position_value.sum()
w_0 = position_value / portfolio_value
target_VaR = 30000 

In [17]:
def opt_weights(weights):
     
     def objective(weights):
          if long:
               var = abs(np.percentile(rt.dot(weights), 100 - conf))
          else:
               var = abs(np.percentile(rt.dot(weights), conf))
          return var
     
     constraints = (
            {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}, 
            {'type': 'eq', 'fun': lambda weights: objective(weights) - target_VaR} 
        )
     
     bounds = [(0, 1) for _ in range(len(positions))]

     initial_guess = np.full(len(positions), 1/len(positions))

     result = minimize(
          fun=objective,  
          constraints=constraints, 
          bounds=bounds,
          x0=initial_guess,
          method='SLSQP')
     
     return result.x

In [22]:
w_opt = opt_weights(w_0)
w_opt

array([0.28886842, 0.20002861, 0.15185341, 0.19560723, 0.16364234])

In [30]:
def rebalanceo(initial_weights, optimal_weights):
     df = pd.DataFrame({
          "Initial Weights": initial_weights*100,
          "Optimal Weights": optimal_weights*100,
          "Stocks difference": np.floor((optimal_weights - initial_weights) * portfolio_value / data.iloc[-1])
     })

     if long:
          df['Buy/Sell'] = df['Stocks difference'].apply(lambda x: 'Buy' if x > 0 else ('Sell' if x < 0 else 'NA'))
     else:
          df['Buy/Sell'] = df['Stocks difference'].apply(lambda x: 'Sell' if x > 0 else ('Buy' if x < 0 else 'NA'))
     
     return df

In [31]:
rebalanceo(w_0, w_opt)

,Initial Weights,Optimal Weights,Stocks difference,Buy/Sell
Ticker,,,,
AAPL,24.57,28.89,385.00,Buy
AMD,18.20,20.00,319.00,Buy
JPM,13.59,15.19,144.00,Buy
LMT,30.64,19.56,-276.00,Sell
TSLA,13.01,16.36,312.00,Buy


In [19]:
def Rebalanceo(data, positions, conf, long, target_VaR):

     valor_posicion = positions * data.iloc[-1]

     weights = []
     for i in range(len(positions)):
          weights.append(valor_posicion.iloc[i]/valor_posicion.sum())
          
     rt_assets = data.pct_change().dropna()

     rt_port = rt_assets.mul(weights).sum(axis=1)

     if long:
          var = np.percentile(rt_port,100-conf)
          cvar = abs(rt_port[rt_port < var].mean())
          var = abs(var)
     else:
          var = np.percentile(rt_port,conf)
          cvar = abs(rt_port[rt_port > var].mean())
          var = abs(var)

     var_m = var * valor_posicion.sum()
     cvar_m = cvar * valor_posicion.sum()

     if var_m < target_VaR:
          resultados_df = pd.DataFrame({
             "Métrica": ["VaR", "CVaR"],
             "Porcentual (%)": [np.round(var * 100, 4), np.round(cvar * 100, 4)],
             "Monetario ($)": [np.round(var_m, 2), np.round(cvar_m, 2)]
          })
          
          resultados_df["Porcentual (%)"] = resultados_df["Porcentual (%)"].map("{:,.4f}".format) 
          resultados_df["Monetario ($)"] = resultados_df["Monetario ($)"].map("{:,.2f}".format) 

          return resultados_df
     else:
        target = abs(target_VaR/valor_posicion.sum())

        def objective(weights, rt_assets, conf):
            rt_port_optim = rt_assets.dot(weights)
            var = abs(np.percentile(rt_port_optim, 100 - conf))
            return var

        def constraint(weights, rt_assets, target, conf):
            rt_port_optim = rt_assets.dot(weights)
            var = abs(np.percentile(rt_port_optim, 100 - conf))
            return var - target 

        constraints = (
            {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}, 
            {'type': 'eq', 'fun': lambda weights: constraint(weights, rt_assets, target, conf)} 
        )

        bounds = [(0, 1) for _ in range(len(positions))]

        initial_weights = np.ones(len(positions)) / len(positions)

        result = minimize(objective, initial_weights, args=(rt_assets, conf), bounds=bounds, constraints=constraints)

        if result.success:
            optim_weights = result.x

            w_diff = [a - b for a, b in zip(optim_weights, weights)]
            money_diff = np.array(w_diff) * valor_posicion.sum()

            position_diff = np.floor(money_diff / data.iloc[-1])
            position_diff_df = pd.DataFrame({
                "ticker": data.keys(), 
                "Diferencia porcentual (%)": np.array(w_diff) * 100, 
                "Diferencia en acciones": position_diff.values
            })

            if long:
                position_diff_df['Compra/Venta'] = position_diff_df['Diferencia en acciones'].apply(lambda x: 'Comprar' if x > 0 else ('Vender' if x < 0 else 'NA'))
            else:
                position_diff_df['Compra/Venta'] = position_diff_df['Diferencia en acciones'].apply(lambda x: 'Vender' if x > 0 else ('Comprar' if x < 0 else 'NA'))

            new_positions = [a + b for a, b in zip(position_diff, positions)]
            new_positions_df = pd.DataFrame({
                "ticker": data.keys(), 
                "Ponderaciones (%)": optim_weights * 100, 
                "Cantidad de acciones": new_positions
            })

            print(f"Para rebalancear el portafolio y tener un VaR monetario de ${target_VaR:,.2f} se necesitan los siguientes cambios en el portafolio")
            display(position_diff_df)
            print(f"Finalmente el portafolio quedaría con las siguientes posiciones")
            display(new_positions_df)
        else:
            return "La optimización no fue exitosa"

In [20]:
data = yf.download(['AAPL', 'AMD', 'JPM', 'LMT', 'TSLA'], start="2020-01-01", end="2023-01-01")['Close'] #No puedo descargar el ajustado

positions = [2193, 3221, 1231, 761, 1211] #Ordenar alfabeticamente

long = True #True o False

conf = 95 #[0, 100]

target_VaR = 30000 

Rebalanceo(data, positions, conf, long, target_VaR)

[*********************100%***********************]  5 of 5 completed

Para rebalancear el portafolio y tener un VaR monetario de $30,000.00 se necesitan los siguientes cambios en el portafolio


,ticker,Diferencia porcentual (%),Diferencia en acciones,Compra/Venta
0,AAPL,-17.42,"-1,555.00",Vender
1,AMD,2.64,467.00,Comprar
2,JPM,-12.60,"-1,142.00",Vender
3,LMT,25.93,643.00,Comprar
4,TSLA,1.45,134.00,Comprar


Finalmente el portafolio quedaría con las siguientes posiciones


,ticker,Ponderaciones (%),Cantidad de acciones
0,AAPL,7.15,638.00
1,AMD,20.84,"3,688.00"
2,JPM,0.99,89.00
3,LMT,56.56,"1,404.00"
4,TSLA,14.46,"1,345.00"


## Ejercicio 3

Investigación conceptual: Rebalanceos

### Inciso a)

- **¿Cuál es el objetivo principal del rebalanceo?**

- **¿Por qué es importante implementarlo de manera sistemática?**


### Inciso b)

- **¿Qué decisiones forman parte de una estrategia de rebalanceo?**

- **¿Cuáles son los métodos principales para realizar rebalanceos de forma sistemática?**


### Inciso c)

- **Además del rendimiento, riesgo y objetivos, ¿qué otro factor puede tener un gran impacto en el desempeño de un portafolio al rebalancearlo?**


### Inciso d)

- **¿Qué recomendaciones propone Vanguard para reducir los costos fiscales en un portafolio?**

- **¿Qué es el tax loss harvesting?**
